# Projeto de Dados - Olist E-commerce
Esse notebook possui os processos de ETL, análises, visualizações e conclusões solicitadas do Dataset Olist E-commerce.

In [102]:
# Importando todas as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## 📥 Importação e Visualização dos Dados

- Leitura dos CSVs para DataFrames
- Obs: A base product_category_name_translation não será utilizada, pois a base products já possui a coluna 'product_category_name'.

In [103]:
df_csv_customers = pd.read_csv('data/olist_customers_dataset.csv')
df_csv_geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
df_csv_order_items = pd.read_csv('data/olist_order_items_dataset.csv')
df_csv_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
df_csv_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
df_csv_orders = pd.read_csv('data/olist_orders_dataset.csv')
df_csv_products = pd.read_csv('data/olist_products_dataset.csv')
df_csv_sellers = pd.read_csv('data/olist_sellers_dataset.csv')

## 🧹 Limpeza e Transformação dos Dados

A limpeza nos dataframes a baixo é feita para evitar duplicidade de dados, selecionar apenas as colunas necessárias 
e evitar erros de leitura. Algumas colunas são renomeadas para facilitar a leitura e o entendimento do código.

- Tratamento de nulos, tipos, duplicatas
- Normalização e seleção de colunas a serem utilizadas

In [104]:
# A base 'orders' possui valores nulos em colunas relevantes de data (order_delivered_carrier_date, order_delivered_customer_date, order_approved_at).
# Para evitar perda de dados, foi criada a coluna 'customer_confirmed_delivery', que assume False
# quando as datas de entrega e aprovação estão ausentes, indicando que o pedido não foi concluído.
def transf_orders(df):
    df['customer_confirmed_delivery'] = (
        df['order_delivered_carrier_date'].isna() &
        df['order_delivered_customer_date'].isna() &
        df['order_approved_at'].isna()
    )
    return df


# Colunas de preço e frete são renomeadas.
# Criação da coluna 'qtd_order_item' com o total de itens, visto que um pedido pode ter mais de uma linha referente ao mesmo produto.
def transf_orders_items(df):
    df = df.groupby(['order_id', 'product_id', 'seller_id', 'price', 'freight_value']).agg(
        qtd_order_item=('order_id', 'count')
    ).reset_index()
    return df.rename(columns={'price': 'product_price', 'freight_value': 'product_shipping_cost'})


# A base de pagamentos pode conter múltiplos registros por pedido, então somamos o valor total pago por pedido.
# Visto que um mesmo pedido pode ter mais de um pagamento, e na análise não é necessário saber o tipo de pagamento ou outras informações.
def transf_payments(df):
    return df.groupby('order_id').agg(total_order_payments =('payment_value', 'sum')).reset_index()


# Caso existam múltiplas avaliações para o mesmo pedido, será considerada apenas a maior nota.
# Além disso, somente as colunas order_id e review_score são mantidas. 
def transf_reviews(df):
    return df.groupby('order_id')['review_score'].max().reset_index(name='order_score_review')


# Mantém apenas ID e categoria dos produtos. Caso a categoria seja nula, será substituída por 'sem_categoria'.
def transf_product(df):
    df['product_category_name'] = df['product_category_name'].fillna('sem_categoria')
    return df[['product_id', 'product_category_name']]

## ✅ Validação de Consistência dos Dados

- Função para identificar nulos, strings vazias e valores negativos.
- Parâmetro opcional para ignorar colunas numéricas específicas ao verificar valores negativos.

In [105]:
def check_inconsistencies(df, nome, ignore_negative_cols=None):
    inconsistencias = []

    # Duplicatas
    duplicadas = df.duplicated().sum()
    if duplicadas > 0:
        inconsistencias.append(f"- Duplicadas: {duplicadas}")

    # Nulos
    nulos = df.isnull().sum()
    nulos = nulos[nulos > 0]
    if not nulos.empty:
        inconsistencias.append("- Valores nulos:\n" + str(nulos))

    # Strings vazias
    vazias = df.select_dtypes(include='object').apply(lambda col: (col == '').sum())
    vazias = vazias[vazias > 0]
    if not vazias.empty:
        inconsistencias.append("- Strings vazias (''):\n" + str(vazias))

    # Valores negativos
    numeric_cols = df.select_dtypes(include='number')
    if ignore_negative_cols:
        numeric_cols = numeric_cols.drop(columns=ignore_negative_cols, errors='ignore')
    negativos = (numeric_cols < 0).sum()
    negativos = negativos[negativos > 0]
    if not negativos.empty:
        inconsistencias.append("- Valores negativos:\n" + str(negativos))

    # Exibe somente se houver alguma inconsistência
    if inconsistencias:
        print(f"\n {nome.upper()}")
        for item in inconsistencias:
            print(item)


In [106]:
# Lembre-se que o df_orders terá dados nulos, pois a coluna customer_confirmed_delivery foi criada para resolver isso.

df_orders = transf_orders(df_csv_orders)
check_inconsistencies(df_orders, 'df_orders')

df_customers = df_csv_customers.drop_duplicates()
check_inconsistencies(df_customers, 'df_customers')

df_geolacation = df_csv_geolocation.drop_duplicates()
check_inconsistencies(df_geolacation, 'df_geolocation', ignore_negative_cols=['geolocation_lat','geolocation_lng'])

df_sellers = df_csv_sellers.drop_duplicates()
check_inconsistencies(df_sellers, 'df_sellers')

df_orders_items = transf_orders_items(df_csv_order_items)
df_orders_items_valid = check_inconsistencies(df_orders_items, 'df_orders_items')

df_payments = transf_payments(df_csv_payments)
df_payments_valid = check_inconsistencies(df_payments, 'df_payments')

df_reviews = transf_reviews(df_csv_reviews)
df_reviews_valid = check_inconsistencies(df_reviews, 'df_reviews')

df_products = transf_product(df_csv_products)
df_products_valid = check_inconsistencies(df_products, 'df_products')


 DF_ORDERS
- Valores nulos:
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
dtype: int64


## Análise Exploratória de Dados (EDA)

## Solução dos Problemas de Negócio
- Análise de Retenção
- Predição de Atraso
- Clustering
- Avaliação de Satisfação

# Visualizações e Dashboards

# Conclusões